In [8]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [9]:

# Your API key
API_KEY = "c682a0eeff664f1b83715929492c7007"

# List of 10 companies/tickers
companies = ["AAPL", "MSFT", "AMZN", "GOOGL", "TSLA",
           "MC.PA", "TTE.PA", "SAN.PA", "AIR.PA", "SU.PA"]

# Date range: last month
end_date = datetime.today()
start_date = end_date - timedelta(days=30)
from_date = start_date.strftime("%Y-%m-%d")
to_date = end_date.strftime("%Y-%m-%d")

# Prepare a list to store all articles
all_articles = []

# Loop through each company
for company in companies:
    url = (
    f"https://newsapi.org/v2/everything?"
    f"q={company}&"
    f"from={from_date}&"
    f"to={to_date}&"
    f"sortBy=publishedAt&"
    f"pageSize=100&"
    f"language=en&"  # <-- filter English articles
    f"apiKey={API_KEY}"
    )

    response = requests.get(url)
    data = response.json()

    # Check if articles exist
    articles = data.get("articles", [])
    for article in articles:
        all_articles.append({
            "Company": company,
            "Title": article.get("title"),
            "Summary": article.get("description"),
            "URL": article.get("url"),
            "PublishedAt": article.get("publishedAt")
        })

# Convert to DataFrame
df = pd.DataFrame(all_articles)

# Display the table
print(df.head(20))  # show first 20 rows
print(f"Total articles fetched: {len(df)}")

   Company                                              Title  \
0     AAPL   Sell Your Winners Before It’s Too Late [Podcast]   
1     AAPL  Belkin、MagSafe/Qi2 25Wワイヤレス充電に対応したiPhone Air/1...   
2     AAPL  Stock market today: Dow, S&P 500, Nasdaq climb...   
3     AAPL  Stock market today: Dow, S&P 500, Nasdaq slip ...   
4     AAPL  Parallels、macOS 26 TahoeやWindows 11 25H2をサポートし...   
5     AAPL          Apple M5チップを搭載した「MacBook Pro」が近く発表されるもよう。   
6     AAPL  Sustained iPhone demand points to 2026 upside ...   
7     AAPL  Dear Salesforce Stock Fans, Mark Your Calendar...   
8     AAPL  Stocks Turn Mixed as Walmart-OpenAI Deal Pushe...   
9     AAPL  Here's How Much $100 Invested In Apple 15 Year...   
10    AAPL   Stocks Fall as US-China Trade Conflict Escalates   
11    AAPL                              rustybt added to PyPI   
12    AAPL  Friday’s selloff broke something in the stock ...   
13    AAPL  Global markets tumble as Beijing imposes new b...   
14    AAPL  Global market

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# FinBERT pré-entraîné pour sentiment financier
MODEL_NAME = "yiyanghkust/finbert-tone"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

/home/zakelm912/hackathon2025/myvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def analyze_sentiment(text):
    # Tokenize le texte
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    # Passage dans le modèle
    outputs = model(**inputs)

    # Softmax pour obtenir les probabilités
    probs = F.softmax(outputs.logits, dim=1)

    # Indices et labels
    sentiment_labels = ["positive", "neutral", "negative"]
    sentiment_idx = torch.argmax(probs, dim=1).item()

    # Probabilité du label choisi
    sentiment_score = probs[0][sentiment_idx].item()

    return sentiment_labels[sentiment_idx], sentiment_score


In [14]:
df = df.dropna(subset=["Summary"])

sentiments = [analyze_sentiment(text) for text in df["Summary"]]

df["Sentiment"] = [s[0] for s in sentiments]
df["SentimentScore"] = [s[1] for s in sentiments]


# Show results
print(df.head(20))

   Company                                              Title  \
0     AAPL   Sell Your Winners Before It’s Too Late [Podcast]   
1     AAPL  Belkin、MagSafe/Qi2 25Wワイヤレス充電に対応したiPhone Air/1...   
2     AAPL  Stock market today: Dow, S&P 500, Nasdaq climb...   
3     AAPL  Stock market today: Dow, S&P 500, Nasdaq slip ...   
4     AAPL  Parallels、macOS 26 TahoeやWindows 11 25H2をサポートし...   
5     AAPL          Apple M5チップを搭載した「MacBook Pro」が近く発表されるもよう。   
6     AAPL  Sustained iPhone demand points to 2026 upside ...   
7     AAPL  Dear Salesforce Stock Fans, Mark Your Calendar...   
8     AAPL  Stocks Turn Mixed as Walmart-OpenAI Deal Pushe...   
9     AAPL  Here's How Much $100 Invested In Apple 15 Year...   
10    AAPL   Stocks Fall as US-China Trade Conflict Escalates   
11    AAPL                              rustybt added to PyPI   
12    AAPL  Friday’s selloff broke something in the stock ...   
13    AAPL  Global markets tumble as Beijing imposes new b...   
14    AAPL  Global market